# Simulating superradiant scattering around Kerr black holes

We use $c = G = 1$.

The Kerr metric is given by:

$$
g_{\mu\nu} = \pmatrix{
	-(1 - \frac{2 M r}{\Sigma}) & 0 & 0 & -\frac{2 M r a \sin^2 \theta}{\Sigma} \\
	0 & \frac{\Sigma}{\Delta} & 0 & 0 \\
	0 & 0 & \Sigma & 0 \\
	-\frac{2 M r a \sin^2 \theta}{\Sigma} & 0 & 0 & (r^2 + a^2 + \frac{2 M r a^2}{\Sigma} \sin^2 \theta) \sin^2 \theta
}
$$

Where:

$$
a = \frac{J}{M}
$$
$$
\Sigma = r^2 + a^2 \cos^2 \theta
$$
$$
\Delta = r^2 - 2Mr + a^2
$$

And the inverse metric is given by:

$$
g^{\mu\nu} = \pmatrix{
	-\frac{1}{\Delta} (r^2 + a^2 + \frac{2 M r a^2}{\Sigma} \sin^2 \theta) &
		0 &
		0 &
		-\frac{2 M r a}{\Delta \Sigma} \\
	0 & \frac{\Delta}{\Sigma} & 0 & 0 \\
	0 & 0 & \frac{1}{\Sigma} & 0 \\
	-\frac{2 M r a}{\Delta \Sigma} &
		0 & 
		0 & 
		\frac{1}{\Delta \sin^2 \theta} (1 - \frac{2 M r}{\Sigma})
}
$$

We aim to find the energy through the following line integral:

$$
E = \frac{Mha}{\pi} \int_C \frac{r}{\Sigma^2} \cdot ds
$$

Where $C$ is a parametrized curve that represents a photon's worldline through spacetime, derived through tracing the geodesic of the photon as it travels within the cavity:

$$
{d^2 x^\mu \over ds^2} =- \Gamma^\mu {}_{\alpha \beta}{d x^\alpha \over ds}{d x^\beta \over ds}
$$

To avoid numerical instabilities associated with extremely small numerical values, a 1 solar mass black hole will be simulated, though black holes used in actual applications are likely to be much smaller, on the order of $9 \times 10^{-22}$ meters. For tracing the geodesic, an RK45 solver is used, with initial conditions:

$$
r_0 = r_E + 99
$$

$$
\theta_0 = \frac{\pi}{2}
$$

$$
\phi_0 = \epsilon
$$

$$
\frac{dr}{d\lambda} = c = 1
$$

$$
\frac{d\theta}{d\lambda} = \frac{d\phi}{d\lambda} = 0
$$

where $r_E$ is the outer ergosphere of the black hole given by $\frac{2M + \sqrt{4M^2 - 4a^2 \cos^2 \theta}}{2}$, and $\epsilon$ is a very small value, intended to ensure that the test photon is emitted nearly vertically above the black hole. Reflections cause a new path to be traced, where the initial conditions $r_0$, $\theta_0$, and $\phi_0$ take the values of $(r, \theta, \phi)$ of the reflection point.

In [33]:
import trimesh
import numpy as np
import matplotlib.pyplot as plt
import torch

In [35]:
def calculate_christoffel(jacob, g_inv, dims):
    # based on https://github.com/AndreaAntoniali/Riemann-tensor-calculator/blob/main/Riemann_Calculations.ipynb
    gamma = np.zeros((dims, dims, dims))
    for beta in range(dims):
        for mu in range(dims):
            for nu in range(dims):
                for alpha in range(dims):
                    gamma[beta,mu,nu] = 1/2 * g_inv[alpha][beta] * (jacob[alpha][mu][nu] + jacob[alpha][nu][mu] - jacob[mu][nu][alpha])
    return gamma

def christoffel_at_point_4d(metric, inverse_metric, t, r, theta, phi, dims):
    coord = torch.tensor([t, r, theta, phi], requires_grad=True)
    g_inv = inverse_metric(coord)
    jacobian = torch.autograd.functional.jacobian(metric, coord, create_graph=True)
    return calculate_christoffel(jacobian, g_inv, dims)

In [36]:
def kerr_metric(coords, M=2e30, a=0.97):
    t = coords[0]
    r = coords[1]
    theta = coords[2]
    phi = coords[3]
    sigma = r ** 2 + a ** 2 * torch.cos(theta) ** 2
    delta = r ** 2 - 2 * M * r + a ** 2
    return torch.tensor([
        [-(1 - (2 * M * r) /sigma), 0., 0., -((2 * M * r * a * torch.sin(theta) ** 2) / sigma)],
        [0., sigma / delta, 0., 0.],
        [0., 0., sigma, 0.],
        [-((2 * M * r * a * torch.sin(theta) ** 2) / sigma), 0., 0., (r ** 2 + a ** 2 + (2 * M * r * a ** 2)/sigma * torch.sin(theta) ** 2) * torch.sin(theta) ** 2]
    ])

def kerr_inverse_metric(coords, M=2e30, a=0.97):
    # Based off https://www.roma1.infn.it/teongrav/onde19_20/kerr.pdf
    t = coords[0]
    r = coords[1]
    theta = coords[2]
    phi = coords[3]
    sigma = r ** 2 + a ** 2 * torch.cos(theta) ** 2
    delta = r ** 2 - 2 * M * r + a ** 2
    return torch.tensor([
        [-1 / delta * (r ** 2 + a ** 2 + (2 * M * r * a ** 2) / sigma * torch.sin(theta) ** 2), 0., 0., -(2 * M * r * a) / (sigma * delta)],
        [0., delta / sigma, 0., 0.],
        [0., 0., 1 / sigma, 0.],
        [-(2 * M * r * a) / (sigma * delta), 0., 0., (delta - a ** 2 * torch.sin(theta) ** 2) / (sigma * delta * torch.sin(theta) ** 2)]
    ])

def kerr_d_dt(X, t):
    # Array of derivatives
    f = np.zeros(X.shape)
    x = X[:4]
    velocities = X[4:]
    f[:4] = velocities
    t, r, theta, phi = x
    Gamma = christoffel_at_point_4d(kerr_metric, kerr_inverse_metric, t, r, theta, phi, 4)

    for i in range(4):
        Gamma[:,:,i] += np.transpose(Gamma[:,:,i])
        Gamma[:,:,i] -= np.diag(np.diag(Gamma[:,:,i]))/2
        f[4 + i] = -velocities @ Gamma[:,:,i] @ velocities
    return f

In [37]:
def outer_ergosphere(M=2e30, theta=np.pi/2, a=0.97):
    return (2 * M + (4 * M ** 2 - 4 * a * (np.cos(theta)) ** 2) ** (1/2)) / 2

In [39]:
def point_xyz_from_spherical(u):
    r = u[1]
    theta = u[2]
    phi = u[3]
    # Convert spherical to cartesian
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    point = [x, y, z]
    return point

In [106]:
def kerr_intersection_rk4(mesh, f, u0, t0=0, tf=100000, h=0.01, debug=False):
    u = u0
    t = t0
    u_array = [u0]
    
    while t < tf:
        print("Value of d/dt, ", f(u, t))
        k1 = h * f(u, t)    
        k2 = h * f(u + 0.5 * k1, t + 0.5*h)
        k3 = h * f(u + 0.5 * k2, t + 0.5*h)
        k4 = h * f(u + k3, t + h)
        u = u + (k1 + 2*(k2 + k3 ) + k4) / 6
        u_array.append(u)
        # As trimesh uses cartesian coordinates
        # to determine intersection, we convert
        # to cartesian coordinates
        point = point_xyz_from_spherical(u)
        if debug:
            print(point)
        if is_on_surface(mesh, np.array(point)):
            if debug:
                print("Found point on surface")
            return u_array, u

In [104]:
def is_on_surface(mesh, point, delta=5, debug=True):
    # Checks if point is on the surface of the mesh
    # with tolerance of 5 meters
    a = mesh.ray.contains_points([point - delta])
    if a and debug:
        print("[DEBUG] Determined point is within cavity")
    else:
        print("[DEBUG] Determined point is not within cavity")
    b = mesh.ray.contains_points([point + delta])
    return a == True and b == False

In [94]:
# Create icosphere for reactor cavity
mesh = trimesh.creation.icosphere(subdivisions=1, radius=outer_ergosphere() + 1e30)

In [95]:
# samples
N = 150

In [96]:
# constants
c = 1.0
r0 = outer_ergosphere() + 99
theta0 = np.pi / 2
phi0 = 1e-6

In [97]:
# initial ray position
initial = [0.0, r0, theta0, phi0, 1.0, c, 0.0, 0.0]

In [100]:
u_spherical = np.array((N + 1) * [initial])

In [101]:
path = [np.array([0, r0, theta0, phi0])]

In [108]:
for i in range(1, N):
    # Find the point of intersection accounting for curving
    # of straight path in Kerr spacetime
    geodesic, intersection_point = kerr_intersection_rk4(mesh, kerr_d_dt, u_spherical[i - 1], h=1e12, debug=True)
    # Find the initial conditions theta0 and phi0 for the next path to trace
    # by finding the reflected vector
    tau, r, theta, phi, v_tau, v_r, v_theta, v_phi = intersection_point
    next_starting_point = [tau, r, theta, phi, v_tau, c, 0.0, 0.0]
    ray_positions_spherical[i] = next_starting_point
    for point in geodesic:
        path.append(point[:4]) # disregard velocity components

Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG

[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value of d/dt,  [1. 1. 0. 0. 0. 0. 0. 0.]
[3.999999999998e+30, 3.999999999999333e+24, 244929359829470.66]
[DEBUG] Determined point is within cavity
Value 

KeyboardInterrupt: 